In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
 
from sklearn.model_selection import train_test_split
 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
 

In [ ]:
df = pd.read_csv('/kaggle/input/us-gasoline-and-diesel-retail-prices-19952021/PET_PRI_GND_DCUS_NUS_W.csv')
df.Date = pd.to_datetime(df.Date, format='%m/%d/%Y')
df

In [ ]:
# sns.pairplot( df ) 

In [ ]:
plt.plot(df.Date, df.A1)
plt.plot(df.Date, df.A2)
plt.plot(df.Date, df.A3)
plt.plot(df.Date, df.R1)
plt.plot(df.Date, df.R2)
plt.plot(df.Date, df.R3)
plt.plot(df.Date, df.M1)
plt.plot(df.Date, df.M2)
plt.plot(df.Date, df.M3)
plt.plot(df.Date, df.P1)
plt.plot(df.Date, df.P2)
plt.plot(df.Date, df.P3)
plt.plot(df.Date, df.D1)
plt.title('All Price')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend(['A1','A2','A3','R1','R2','R3','M1','M2','M3','P1','P2','P3','D1'])
plt.figure(figsize=(15,15))
plt.show()

In [ ]:
df['avg'] = df.A1 + df.A2 + df.A3 + df.R1 + df.R2 + df.R3 + df.M1 + df.M2 + df.M3 + df.P1 + df.P2 + df.P3 + df.D1
df.avg = df.avg / 13
df

In [ ]:
df.plot(x='Date',y='avg')

In [ ]:
df['Rol5'] = df.avg.rolling(5).mean()
df['Rol20'] = df.avg.rolling(20).mean()
df['Rol3'] = df.avg.rolling(3).mean()
df = df.dropna()
df

In [ ]:
df.plot(x='Date',y='Rol5')

In [ ]:
df.plot(x='Date',y='Rol3')

In [ ]:
df.plot(x='Date',y='Rol20')

In [ ]:
df['Rol50'] = df.avg.rolling(100).mean()
df.dropna().plot(x='Date',y='Rol50')

In [ ]:
plt.plot(df.Date, df.Rol50)
plt.plot(df.Date, df.avg)
plt.show()

In [ ]:
dateTest = '2012-01-04'
# df.loc[df['Host'] == 'a', 'Port'].item()
y1 = df.loc[df.Date < dateTest, 'Rol50'].tail(1).item()
y2 = df.loc[df.Date >= dateTest, 'Rol50'].head(1).item()
m = y2 - y1
pred = df.loc[df.Date >= dateTest, 'Rol50'].head(2).tail(1).item()
df['slopePred'] = df.Rol50 + m
df

In [ ]:
df.slopePred = df.Rol50.diff()
df

In [ ]:
df.slopePred = df.Rol50 + df.slopePred

In [ ]:
plt.plot(df.Date, df.Rol50)
plt.plot(df.Date, df.avg)
plt.plot(df.Date, df.slopePred)
plt.show()

In [ ]:
df.Rol50.shift(1)

In [ ]:
df.Rol50

In [ ]:
oilPred = df[['Date','avg','Rol50']]
oilPred = oilPred.dropna()
oilPred

In [ ]:
oilPred.plot(x='Date',y='avg')

In [ ]:
oilPred.plot(x='Date',y='Rol50')

In [ ]:
oilPred['Up'] = oilPred.Rol50.shift(1)
# oilPred['Low'] = oilPred.Rol50.shift(-1)
oilPred

In [ ]:
oilPred['Dev1'] = oilPred.Rol50 - oilPred.Up
# oilPred.slope = oilPred.slope / 2
oilPred['pred'] = oilPred.Rol50 + oilPred.Dev1
oilPred['pred'] = oilPred['pred'].shift(1)
oilPred['error'] = oilPred.Rol50 - oilPred.pred
oilPred

In [ ]:
oilPred.describe()

0.005182

4.679743e-04 = 0.000468

In [ ]:
oilModelTest = oilPred[['Date','avg','Rol50']]
oilModelTest['Shift1'] = oilModelTest.Rol50.shift(1)
oilModelTest['Shift2'] = oilModelTest.Rol50.shift(2)
oilModelTest['Shift3'] = oilModelTest.Rol50.shift(3)
oilModelTest['Shift4'] = oilModelTest.Rol50.shift(4)
oilModelTest['Shift5'] = oilModelTest.Rol50.shift(5)
oilModelTest = oilModelTest.dropna()
oilModelTest

In [ ]:
model = LinearRegression()
x = oilModelTest[['Shift1','Shift2','Shift3','Shift4','Shift5']]
y = oilModelTest.Rol50
train_x, test_x, train_y, test_y = train_test_split(x, y)
model.fit(train_x, train_y)
predictions = model.predict( test_x )

mean_absolute_error(test_y, predictions)

0.00035

In [ ]:
model = LinearRegression()
x = oilModelTest[['Shift1']]
y = oilModelTest.Rol50
train_x, test_x, train_y, test_y = train_test_split(x, y)
model.fit(train_x, train_y)
predictions = model.predict( test_x )

mean_absolute_error(test_y, predictions)